# Pop, Rap or Heavy Metal?
## Can we tell by looking at song lyrics?

| Pop | Rap | Heavy Metal |
| :---: | :---: | :---: |
| ![](https://imgix.ranker.com/user_node_img/108/2153992/original/taylor-swift-recording-artists-and-groups-photo-u233?w=125&h=125&fit=crop&crop=faces&q=60&fm=pjpg) | ![](https://imgix.ranker.com/user_node_img/3107/62130966/original/kendrick-lamar-photo-u37?w=125&h=125&fit=crop&crop=faces&q=60&fm=pjpg) | ![](https://imgix.ranker.com/user_node_img/88/1742531/original/ozzy-osbourne-recording-artists-and-groups-photo-u41?w=125&h=125&fit=crop&crop=faces&q=60&fm=pjpg) |
| We could leave the Christmas lights up 'til January<br>This is our place, we make the rules<br>And there's a dazzling haze, a mysterious way about you, dear<br>Have I known you twenty seconds or twenty years? |K-Dot, pick up the phone, nigga<br>Every time I call, it's going to voicemail<br>Don't tell me they got you on some weirdo rap shit, nigga?<br>No socks and skinny jeans and shit, ha<br>Call me on Shaniqua's phone! | Times have changed and times are strange<br>Here I come, but I ain't the same<br>Mama, I'm coming home<br>Times gone by seems to be<br>You could have been a better friend to me<br>Mama, I'm coming home|



## Can we teach a machine to tell the difference?
Let's give it a try!



## First things first
Let us plug in all the libraries we need and do all intial setup. We also declare global parameters here, which we are going to use during data preparation and model training.

In [ ]:
# Import general purpose packages
import numpy as np 
import pandas as pd 
import sys
import json
import os
import re

# This is what we are using for data preparation and ML part (thanks, Rafal, for great tutorial)
from sklearn.feature_extraction import text      
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
# Different ML models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB

#Global parameters
SONGS_PER_GENRE = 10000
SONGS_PER_TRAINING = 1000
SONGS_PER_TESTING = 100

#we need to cleanse lyrics to remove special characters and garbage
#this function cleanses single string
def cleanse (text):
    result = re.sub('[^a-zA-Z0-9 ]', ' ', text)
    return result



## Now let's load some data
I am utilizing several data sets all having their unique format, while ultimately I want a dataset with just 4 columns:
* Genre
* Artist
* Song title
* Song lyrics 

I have to approach each data set differently.

### Pop songs
I have a nice and convenient dataset here

In [ ]:
partists = pd.read_csv('/kaggle/input/scrapped-lyrics-from-6-genres/artists-data.csv') #load the list of artists
psongs = pd.read_csv('/kaggle/input/scrapped-lyrics-from-6-genres/lyrics-data.csv') #load the list of songs

In [ ]:


pop_artists = partists[partists['Genre']=='Pop'] # filter artists by genre
pop_songs = pd.merge(psongs, pop_artists, how='inner', left_on='ALink', right_on='Link') #inner join of pop artists with songs to get only songs by pop artists
pop_songs = pop_songs[['Genre', 'Artist', 'SName', 'Lyric']].rename(columns={'SName':'Song'})#leave only columns of interest and rename some of them.
pop_songs = pop_songs.dropna() # Remove incomplete records, cleanse lyrics
pop_songs = pop_songs[pop_songs['Lyric']!='Instrumental'].head(SONGS_PER_GENRE).applymap(cleanse) #Remove instrumental compositions  and limit the size of final dataset
pop_songs.head()

### Rap songs
Things are a little bit more complicated. I am going to use a dataset of lyrics and join it with tracks database from spotify to filter songs by genre.

In [ ]:
#raw data
songs = pd.read_csv('/kaggle/input/150k-lyrics-labeled-with-spotify-valence/labeled_lyrics_cleaned.csv') #load the list of songs
artists = pd.read_csv('/kaggle/input/ultimate-spotify-tracks-db/SpotifyFeatures.csv') #load artists by genere spotify database

In [ ]:
rap_artists=artists[artists['genre']=='Rap'][['genre','artist_name']].drop_duplicates() #extract only artist names and genres, deduplicate the list
rap_songs = pd.merge(songs, rap_artists, how='inner', left_on='artist', right_on='artist_name') #inner join of rap artists with songs to get only songs by rap artists
rap_songs = rap_songs[['genre', 'artist', 'song', 'seq']].rename(columns={'genre':'Genre', 'artist':'Artist', 'song':'Song','seq':'Lyric'}) #leave only columns of interest and rename some
rap_songs = rap_songs.dropna().head(SONGS_PER_GENRE).applymap(cleanse) #remove incomplete records and limit dataset size
rap_songs.head()

### Metal songs
This is the hardest part of data preparation. I have a massive dataset of metal lyrics, but it is stored as text files in folder structure, so data transformation is going to be totally different.

In [ ]:
# since I don't need 250K songs for my analysis and I don't want to wait forever for data to load 
# I am going to build a little function, which is going to return required number of songs
def list_n_metal_songs (limit):
    # intputs:
    # limit - number of songs to return
    # outputs:
    # the list of file paths to songs of required length
    counter = 1
    result = []
    # lets repurpose kaggle boilerplate code to obtain the list of all files with lyrics
    for dirname, _, filenames in os.walk('/kaggle/input/large-metal-lyrics-archive-228k-songs/metal_lyrics'):
        for filename in filenames:
            result.append(os.path.join(dirname, filename))
            #some really bad programming pattern here
            if counter >= limit: return result
            else: counter+=1

# let's get limited set of songs                
metal_files = list_n_metal_songs(SONGS_PER_GENRE)

# let's build our dataset 
artist = []
song = []
lyric = []

#we are going to iterate through the list of files
for path in metal_files:
    # now, since the format of the path is known: <letter>/<artist>/<album>/<track #>.<track name>.txt we can obtain all metadata from file path
    p = path.split('/')
    s = p[8].split('.')
    artist.append(p[6])
    song.append(s[1].strip())
    #we can also open and read file content
    f = open (path, 'r')
    lyric.append(f.read())
    f.close()

#finally, we can assemble metal songs dataset
metal_songs = pd.DataFrame({'Genre':'Metal', 'Artist' : artist, 'Song' : song,'Lyric' : lyric}).applymap(cleanse)
metal_songs.head()

### Summary of data load
We now have datasets of songs in three genres, which we can use to build genre detector

In [ ]:
print("Pop -",len(pop_songs), "| Rap -", len(rap_songs), "| Metal -",len(metal_songs))

## Data preparation for analysis
We are going to create "bag of words" representation of songs in our datasets. We are going to be using ~half of each dataset for model training

In [ ]:
#prepare training dataset by taking equal scoops from each dataset
training_data = pd.concat([pop_songs.head(SONGS_PER_TRAINING), rap_songs.head(SONGS_PER_TRAINING), metal_songs.head(SONGS_PER_TRAINING)])

In [ ]:
#Creating vocabulary
cv = CountVectorizer(strip_accents='ascii', lowercase=True, stop_words='english', analyzer='word') #create the CountVectorizer object
cv.fit(training_data['Lyric'].values) #fit into our dataset

#Creating bag of words representations
bow = cv.transform(training_data['Lyric'].values) 
print(bow.shape[0], 'samples x ',bow.shape[1],'words in vocabulary' )

## Model training
Now we are going to train several ML models (sounds much fancier than it is)

In [ ]:
#Create the machine learning classifier object
models = {'Logistic Regression':LogisticRegression(max_iter=500), 'Linear SVC':LinearSVC(max_iter=10000),'Decision Tree': DecisionTreeClassifier(), 'Gradient Descent': SGDClassifier()}
# things that didn't work 'Nu SVC': NuSVC(), 'SVC': SVC(), 'Naive Bayes': GaussianNB()
for the_model in models.keys():
    print ("Training", the_model)
    models[the_model].fit(bow.toarray(), training_data['Genre'])


## Accuracy Evaluation
Let's use the rest of the dataset to evaluate prediction accuracy

In [ ]:
#prepare data for testing
test_data=pd.concat([pop_songs.iloc[SONGS_PER_TRAINING:SONGS_PER_TRAINING+SONGS_PER_TESTING], rap_songs.iloc[SONGS_PER_TRAINING:SONGS_PER_TRAINING+SONGS_PER_TESTING], metal_songs.iloc[SONGS_PER_TRAINING:SONGS_PER_TRAINING+SONGS_PER_TESTING]])

#convert lyrics to bag of word representation
test_bow = cv.transform(test_data['Lyric'].values) 

In [ ]:
#predict labels for all the sentences
accuracy = {}
pg = test_data[['Artist', 'Song','Genre']]
for the_model in models.keys():
    print("Evaluating",the_model)
    pred_genre= models[the_model].predict(test_bow.toarray())
    pg[the_model]=pred_genre
    accuracy[the_model]=accuracy_score(test_data['Genre'],pred_genre)
print ("---Accuracy Scores---")
print(accuracy)

# Lessons Learned
1. Data preparation is 75% of effort
2. Data cleansing cannot be finished, it only can be stopped. I still have songs not in english and plenty of garbage in my lyrics
2. Model selection and tune up is more an art than methodology, hence high salaries of data scientists and my low score on "Check your understanding 5"
3. Every model has its own bias. Some of them lean towards Metal, some towards Rap when they cannot determine the genre. Looks funny sometimes
4. Lengthier texts do not always provide more accurate classification
5. More data does not guarantee better accuracy (I wonder why...)

# Try it yourself!
Run the snippet of code down below and put any piece of lyrics into the input field. The panel of artificial experts will rate your song as Pop, Rap or Metal.

In [ ]:
#get user input
user_input = input()

#transform it to bag of words
user_bow = cv.transform([cleanse(user_input)]) 

#run against trained models
for the_model in models.keys():
    print(the_model, models[the_model].predict(user_bow))
